In [1]:
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, BatchNormalization, LSTM, Dense, concatenate, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam, SGD
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model

import numpy as np
import random
import matplotlib.pyplot as plt
import os
import cv2
import json

Using TensorFlow backend.


In [2]:
from model.input_fn import train_input_fn
from model.input_fn import test_input_fn
from model.model_fn import model_fn
from model.utils import Params

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13264027311570553119
]


# 1. Define loss & base network
- 참고 소스 https://omoindrot.github.io/triplet-loss

In [ ]:
def batch_hard_triplet_loss(labels, embeddings, margin, squared=False):
    """Build the triplet loss over a batch of embeddings.

    For each anchor, we get the hardest positive and hardest negative to form a triplet.

    Args:
        labels: labels of the batch, of size (batch_size,)
        embeddings: tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.

    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    # Get the pairwise distance matrix
    pairwise_dist = _pairwise_distances(embeddings, squared=squared)

    # For each anchor, get the hardest positive
    # First, we need to get a mask for every valid positive (they should have same label)
    mask_anchor_positive = _get_anchor_positive_triplet_mask(labels)
    mask_anchor_positive = tf.to_float(mask_anchor_positive)

    # We put to 0 any element where (a, p) is not valid (valid if a != p and label(a) == label(p))
    anchor_positive_dist = tf.multiply(mask_anchor_positive, pairwise_dist)

    # shape (batch_size, 1)
    hardest_positive_dist = tf.reduce_max(anchor_positive_dist, axis=1, keepdims=True)
    tf.summary.scalar("hardest_positive_dist", tf.reduce_mean(hardest_positive_dist))

    # For each anchor, get the hardest negative
    # First, we need to get a mask for every valid negative (they should have different labels)
    mask_anchor_negative = _get_anchor_negative_triplet_mask(labels)
    mask_anchor_negative = tf.to_float(mask_anchor_negative)

    # We add the maximum value in each row to the invalid negatives (label(a) == label(n))
    max_anchor_negative_dist = tf.reduce_max(pairwise_dist, axis=1, keepdims=True)
    anchor_negative_dist = pairwise_dist + max_anchor_negative_dist * (1.0 - mask_anchor_negative)

    # shape (batch_size,)
    hardest_negative_dist = tf.reduce_min(anchor_negative_dist, axis=1, keepdims=True)
    tf.summary.scalar("hardest_negative_dist", tf.reduce_mean(hardest_negative_dist))

    # Combine biggest d(a, p) and smallest d(a, n) into final triplet loss
    triplet_loss = tf.maximum(hardest_positive_dist - hardest_negative_dist + margin, 0.0)

    # Get final mean triplet loss
    triplet_loss = tf.reduce_mean(triplet_loss)

    return triplet_loss

# 2. Load data

In [7]:
#data_dir = '../../../../../Documents/pshopper/mmfashion/data/Attr_Predict/Img_fortest/'
#on my laptop
data_dir = '../../../../../../Documents/pshopper/mmfashion/data/Attr_Predict/Img_fortest/'

In [20]:
BATCH_SIZE = 64
IMG_DIM = 28
#NB_CLASSES = 5620
NB_CLASSES = 5

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                        data_dir,  # this is the target directory
                        target_size=(IMG_DIM, IMG_DIM),
                        batch_size=BATCH_SIZE)

Found 273 images belonging to 5 classes.


In [21]:
batch = train_generator.next()
images = batch[0]
labels = batch[1]

# images = images.reshape(-1, 150, 150)
labels = np.array([np.argmax(x) for x in labels])

In [22]:
images.shape

(64, 28, 28, 3)

In [23]:
labels.shape

(64,)

# 3. Create model & Train

In [39]:
def input_func_gen():
    ds = tf.data.Dataset.from_generator(lambda: train_generator,
                     output_types=(tf.float32, tf.float32),
                     output_shapes=([None, IMG_DIM, IMG_DIM, 3], [None, NB_CLASSES]))
    it = ds.make_one_shot_iterator()        #https://stackoverflow.com/questions/52636127/how-to-use-keras-generator-with-tf-data-api
    features, labels = it.get_next()
    labels = tf.argmax(labels, axis=1)
    return features, labels

In [40]:
features, labels = input_func_gen()

(<tf.Tensor 'IteratorGetNext:0' shape=(?, 28, 28, 3) dtype=float32>,
 <tf.Tensor 'ArgMax:0' shape=(?,) dtype=int64>)

In [41]:
model_dir = 'experiments/batch_hard'

json_path = os.path.join(model_dir, 'params.json')

params = Params(json_path)

In [42]:
K.clear_session()

In [43]:
tf.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()

In [44]:
tf.executing_eagerly()

False

In [ ]:
tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.INFO)

# Define the model
tf.logging.info("Creating the model...")
config = tf.estimator.RunConfig(tf_random_seed = 230,
                                model_dir = model_dir,
                                save_summary_steps = params.save_summary_steps)
estimator = tf.estimator.Estimator(model_fn, params=params, config=config)

# Train the model
tf.logging.info("Starting training for {} epoch(s).".format(params.num_epochs))
estimator.train(input_func_gen)

INFO:tensorflow:Creating the model...
INFO:tensorflow:Using config: {'_model_dir': 'experiments/batch_hard', '_tf_random_seed': 230, '_save_summary_steps': 50, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002238AF15198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Starting training for 10 epoch(s

INFO:tensorflow:loss = 0.0, step = 15369 (9.619 sec)
INFO:tensorflow:global_step/sec: 11.385
INFO:tensorflow:loss = 0.0, step = 15469 (8.778 sec)
INFO:tensorflow:global_step/sec: 12.1669
INFO:tensorflow:loss = 0.0, step = 15569 (8.220 sec)
INFO:tensorflow:global_step/sec: 11.8393
INFO:tensorflow:loss = 0.0, step = 15669 (8.444 sec)
INFO:tensorflow:global_step/sec: 12.9077
INFO:tensorflow:loss = 0.0, step = 15769 (7.747 sec)
INFO:tensorflow:global_step/sec: 11.7081
INFO:tensorflow:loss = 0.0, step = 15869 (8.542 sec)
INFO:tensorflow:global_step/sec: 12.8845
INFO:tensorflow:loss = 0.0, step = 15969 (7.761 sec)
INFO:tensorflow:global_step/sec: 12.7082
INFO:tensorflow:loss = 0.0, step = 16069 (7.869 sec)
INFO:tensorflow:global_step/sec: 12.7957
INFO:tensorflow:loss = 0.0, step = 16169 (7.815 sec)
INFO:tensorflow:global_step/sec: 11.142
INFO:tensorflow:loss = 0.0, step = 16269 (8.975 sec)
INFO:tensorflow:global_step/sec: 11.5609
INFO:tensorflow:loss = 0.0, step = 16369 (8.651 sec)
INFO:tens

INFO:tensorflow:global_step/sec: 10.6193
INFO:tensorflow:loss = 0.0, step = 24069 (9.417 sec)
INFO:tensorflow:global_step/sec: 13.1533
INFO:tensorflow:loss = 0.0, step = 24169 (7.600 sec)
INFO:tensorflow:global_step/sec: 11.5675
INFO:tensorflow:loss = 0.0, step = 24269 (8.647 sec)
INFO:tensorflow:global_step/sec: 12.5761
INFO:tensorflow:loss = 0.0, step = 24369 (7.952 sec)
INFO:tensorflow:global_step/sec: 11.0621
INFO:tensorflow:loss = 0.0, step = 24469 (9.040 sec)
INFO:tensorflow:global_step/sec: 10.7502
INFO:tensorflow:loss = 0.0, step = 24569 (9.302 sec)
INFO:tensorflow:global_step/sec: 11.4749
INFO:tensorflow:loss = 0.0, step = 24669 (8.714 sec)
INFO:tensorflow:global_step/sec: 11.6225
INFO:tensorflow:loss = 0.0, step = 24769 (8.604 sec)
INFO:tensorflow:global_step/sec: 11.4801
INFO:tensorflow:loss = 0.0, step = 24869 (8.711 sec)
INFO:tensorflow:global_step/sec: 12.1301
INFO:tensorflow:loss = 0.0, step = 24969 (8.244 sec)
INFO:tensorflow:global_step/sec: 12.0354
INFO:tensorflow:los

In [ ]:
tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.INFO)

# Define the model
tf.logging.info("Creating the model...")
config = tf.estimator.RunConfig(tf_random_seed = 230,
                                model_dir = model_dir,
                                save_summary_steps = params.save_summary_steps)
estimator = tf.estimator.Estimator(model_fn, params=params, config=config)

# Train the model
tf.logging.info("Starting training for {} epoch(s).".format(params.num_epochs))
estimator.train(input_func_gen)

In [182]:
num_channels = 32
channels = [num_channels, num_channels * 2]

In [172]:
for i, c in enumerate(channels):
    print(i, c)

0 32
1 64


In [ ]:
base_network.summary()    #base_network (즉 인코더) 서머리 보기
plot_model(base_network, to_file='./CNN/base_network.png', show_shapes=True, show_layer_names=True)    #base_network 도식화 + 저장

In [ ]:
# Evaluate the model on the test set
tf.logging.info("Evaluation on test set.")
res = estimator.evaluate(lambda: test_input_fn(args.data_dir, params))
for key in res:
    print("{}: {}".format(key, res[key]))

In [ ]:
model.summary()     #전체 모델 서머리 보기
plot_model(model, to_file='./CNN/cnn_model.png', show_shapes=True, show_layer_names=True)     #전체 모델 도식화 + 저장

In [ ]:
# Training the model
H = model.fit([a, p, n], y_dummie, batch_size=50, epochs=10)

In [ ]:
#결과 확인
plt.plot(H.history['loss'])
plt.savefig('./CNN/loss_graph.png')
# plt.legend(['loss'], loc = 'upper left')
plt.show()

# 5. Save model & weights

In [ ]:
model_dir = './CNN/encoder.json'
model_weights_dir = './CNN/encoder.h5'

In [ ]:
#Save model
model_json = model.to_json()
with open(model_dir, "w") as json_file : 
    json_file.write(model_json)

#Save weights
model.save_weights(model_weights_dir)
print("Saved model to disk")